In [2]:
#!pip3 install "psycopg[binary]"

In [19]:
import config
import psycopg
import pytz
import pandas as pd
import numpy as np
from psycopg.rows import dict_row

In [4]:
def pgDictToConn(secretDict):
    pgStrs = []
    for key in secretDict:
        pgStrs.append('{}={}'.format(key, secretDict[key]))
    return ' '.join(pgStrs)


In [5]:
#Init
pgConnStr = pgDictToConn(config.pgSecrets)

In [31]:
def getTicker(symbol):
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''SELECT * FROM alpaca_minute WHERE symbol = %s'''
            data = (symbol, )
            result = cur.execute(stmt, data).fetchall()
            desc = cur.description
            cols = [col[0] for col in desc]
            conn.commit()
    df = pd.DataFrame(data=result, columns=cols)
#     df['datetime'] = df['datetime'].dt.tz_convert('America/New_York')
    df['datetime'] = df['datetime'].dt.tz_convert('America/New_York')
    return df #pd.DataFrame(data=result, columns=cols)

In [29]:
def getTickerList():
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''SELECT DISTINCT symbol FROM alpaca_minute ORDER BY symbol'''
            result = cur.execute(stmt).fetchall()
            conn.commit()
            return [row['symbol'] for row in result]

In [9]:
#Get List of all tickers
len(getTickerList())

1

In [36]:
#Sample 10 random stocks
tickers = getTickerList()
random_pick = list(np.random.choice(tickers, 10 if len(tickers) > 10 else  len(tickers)))
random_pick

['AAPL']

In [33]:
getTicker(random_pick[0])

,id,date,symbol,open,close,high,low,trade_count,vol,vwap,datetime
0,1,2022-09-15,AAPL,154.2550,154.1401,154.2650,154.10,1885.0,202191.0,154.175270,2022-09-15 10:26:00-04:00
1,4,2022-09-15,AAPL,154.1499,153.8980,154.1588,153.87,4838.0,441850.0,153.982115,2022-09-15 10:27:00-04:00
2,5,2022-09-15,AAPL,153.8900,153.9139,153.9300,153.77,2720.0,302522.0,153.849119,2022-09-15 10:28:00-04:00
3,6,2022-09-15,AAPL,153.9050,153.8107,153.9300,153.75,2480.0,288487.0,153.848830,2022-09-15 10:29:00-04:00
4,7,2022-09-15,AAPL,153.8100,153.9000,153.9800,153.73,2704.0,257037.0,153.857872,2022-09-15 10:30:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...
47938,47941,2023-03-13,AAPL,150.5000,150.5000,150.5000,150.50,18.0,367.0,150.543365,2023-03-13 16:52:00-04:00
47939,47942,2023-03-13,AAPL,150.5600,150.6300,150.6300,150.56,11.0,1318.0,150.590827,2023-03-13 16:53:00-04:00
47940,47943,2023-03-13,AAPL,150.6200,150.6200,150.6200,150.62,29.0,679.0,150.592695,2023-03-13 16:54:00-04:00
47941,47944,2023-03-13,AAPL,150.5700,150.5700,150.5800,150.43,40.0,2757.0,150.513714,2023-03-13 16:55:00-04:00


In [34]:
#Get Specific Ticker
#getTicker('AAPL')